<a href="https://colab.research.google.com/github/jjpvanderburg-sys/oseo-lansingerland-water/blob/main/Lansingerland_Slotenkaart_NDWI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openeo geopandas shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.8/337.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.12.0
    Uninstalling xarray-2025.12.0:
      Successfully uninstalled xarray-2025.12.0


In [ ]:
import openeo

# Verbinden met de openEO Platform backend
conn = openeo.connect("openeo.cloud").authenticate_oidc()

print("✅ Verbonden met openEO:", conn)

Authenticated using refresh token.
✅ Verbonden met openEO: <Connection to 'https://openeocloud.vito.be/openeo/1.0.0/' with OidcBearerAuth>


In [ ]:
from google.colab import files

# ⬆️ Upload je KML-bestand (selecteer het vanaf je computer)
uploaded = files.upload()

Saving Lansingerland.kml to Lansingerland.kml


In [ ]:
import geopandas as gpd

# 🔧 Zet hier exact de naam van je KML-bestand
kml_path = "/content/Lansingerland.kml"

# 📥 KML inlezen
gdf = gpd.read_file(kml_path, driver="KML")

# 🔍 Controle: wat zit erin?
print("Aantal features:", len(gdf))
print("Kolommen:", gdf.columns)

# 🗺️ Toon de geometrie (gemeentegrens)
gdf

Aantal features: 1
Kolommen: Index(['id', 'Name', 'description', 'timestamp', 'begin', 'end',
       'altitudeMode', 'tessellate', 'extrude', 'visibility', 'drawOrder',
       'icon', 'snippet', 'geometry'],
      dtype='object')


,id,Name,description,timestamp,begin,end,altitudeMode,tessellate,extrude,visibility,drawOrder,icon,snippet,geometry
0,ID_00000,GM1621,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",NaT,NaT,NaT,clampToGround,-1,0,-1,NaN,None,,"MULTIPOLYGON Z (((4.48356 51.97433 0, 4.48364 ..."


In [ ]:
# 📐 Zet geometrie om naar GeoJSON
gdf = gdf.to_crs("EPSG:4326")  # Zorg dat het in WGS84 staat
geometry = gdf.geometry.iloc[0]  # Neem de eerste (en enige) geometrie

geojson_geometry = geometry.__geo_interface__  # openEO-ready
print("✅ GeoJSON polygoon klaar voor gebruik in openEO.")

✅ GeoJSON polygoon klaar voor gebruik in openEO.


In [ ]:
# 📦 Ophalen van Sentinel-2 data voor Lansingerland
cube = (
    conn.load_collection(
        "SENTINEL2_L2A",
        spatial_extent=geojson_geometry,
        temporal_extent=["2025-01-01", "2025-12-31"],
        bands=["B03", "B08"],  # Green + NIR
    )
)

print("✅ Sentinel-2 data cube aangemaakt.")

✅ Sentinel-2 data cube aangemaakt.


In [ ]:
from openeo.processes import normalized_difference

green = cube.band("B03")
nir = cube.band("B08")

ndwi_cube = normalized_difference(green, nir)

print("✅ NDWI‑datacube correct aangemaakt")

✅ NDWI‑datacube correct aangemaakt


In [ ]:
# 📦 Binary watermasker per tijdstap maken op basis van NDWI
# Water = 1 als NDWI > 0.2

water_mask_cube = ndwi_cube > 0.2

print("✅ Watermaskers per tijdstap zijn aangemaakt (0 = geen water, 1 = water)")

✅ Watermaskers per tijdstap zijn aangemaakt (0 = geen water, 1 = water)


✅ Is een nieuw raster nodig?

Ja, maar we hoeven dat niet handmatig te bouwen.
We reduceren de tijdsdimensie met een sum-operatie:

In [ ]:
# 📊 Tellen hoe vaak water werd gedetecteerd per pixel in 2025
water_count = water_mask_cube.reduce_dimension(
    dimension="t",  # tijdsdimensie
    reducer="sum"   # tel het aantal keren '1' = water
)

print("✅ Waterfrequentie per pixel is berekend.")

✅ Waterfrequentie per pixel is berekend.


In [ ]:
from openeo.processes import if_, int as p_int

# ✅ Zet True/False om naar 1/0 → betere typeherkenning
water_mask_int = water_mask_cube.apply(lambda x: p_int(if_(x, 1, 0)))

print("✅ Boolean → integer conversie uitgevoerd.")

✅ Boolean → integer conversie uitgevoerd.


In [ ]:
# ✅ Reduce over tijd → tel aantal keer dat water is gedetecteerd
water_count = water_mask_int.reduce_dimension(dimension="t", reducer="sum")

# ✅ Sla op als GeoTIFF
export_cube = water_count.save_result(format="GTiff")

# ✅ Start batch job
job = conn.create_job(export_cube)
job.start_and_wait()

# ✅ Downloaden
job.download_results("results/")
print("✅ GeoTIFF opgeslagen in map: results/")

0:00:00 Job 'vito-j-2601191413404969be7ea73716e45f08': send 'start'


OpenEoApiError: [500] Internal: Unexpected error during 'subtract': Exception('Datatype unknown.'). The process had these arguments: {'x': DryRunDataCube(metadata=CollectionMetadata(dimension_names=['x', 'y', 't'])), 'y': DryRunDataCube(metadata=CollectionMetadata(dimension_names=['x', 'y', 't']))}  (Upstream ref: 'r-260119141342421f9e8ce7031934b8c9') (ref: r-260119141340470abe48811a552ece31)